In [53]:
filename = './data/axaf_mrh_ddea_freq_tot.csv'
features = [
    'ddea_quant_cm_10',
        'HAB_nb_pieces',
        'HAB_qual',
        'HAB_anclg',
    'CLI_age',
    #'POL_mtcapass',
    'HAB_habit',
        'CLI_sex',
    #'POL_tr_tx_objv',
    'POL_fract',
    'annee',
    'CLI_nb_enfant',
]   
    
features = [    
    'EXT_dens_pop',
    'EXT_part_respr_1p',
    'EXT_part_respr_5p',
    'EXT_part_secocc',
    'EXT_part_surf_inf40',
    'EXT_part_surf_plus100',
    'EXT_part_rev_15K29k',
    'EXT_part_men_lochlm',
    'EXT_part_men_pror',
    'EXT_part_pop_18a24a',
    'EXT_part_pop_65a79a',
    'EXT_part_respr_chauf_aut',
    'EXT_part_respr_chauf_col',
    'EXT_part_respr_chauf_ind',
    'EXT_part_respr_chauf_indelec',
    'EXT_part_repsr_sdb1',
    'EXT_part_repsr_sdb2',
    'EXT_part_lgt_co',
    'EXT_part_lgt_vac',
    'EXT_part_pop_15dipsup',
    'EXT_part_pop_15ssdip',
    'EXT_gdeville',
    'AUTV_prix_sra_max',
    'AUTV_prix_sra_min',
    'AUTV_rm',
    'AUTV_segment_max',
    'AUTV_segment_min',
    'CLI_banque',
    'POL_cdopmrh',
    'POL_fract',
    'POL_cdregion',
    'HAB_cdresid',
    'CLI_sex',
    'POL_distrib',
    'AUTV_energie_es',
    'AUTV_energie_go',
    'AUTV_k8000_n',
    'AUTV_k8000_o',
    'AUTV_prix_sra_ad',
    'AUTV_prix_sra_ef',
    'AUTV_prix_sra_gi',
    'AUTV_prix_sra_jl',
    'AUTV_prix_sra_mn',
    'AUTV_prix_sra_supo',
    'AUTV_segment_0',
    'AUTV_segment_a',
    'AUTV_segment_b',
    'AUTV_segment_h',
    'AUTV_segment_m1',
    'AUTV_segment_m2',
    'AUTV_nb_rc_12',
    'AUTV_nb_rc_34',
    'AUTV_nb_rc_56',
    'AUTV_nb_rc_78',
    'AUTV_nb_rc_sup9',
    'AUTV_nb_vol_ab',
    'AUTV_nb_vol_cd',
    'AUTV_nb_vol_ef',
    'AUTV_nb_vol_gh',
    'AUTV_nb_vol_ij',
    'AUTV_nb_vol_supk',
    'HAB_inser',
    'CLI_indice',
    'AUTA_tot1a',
    'AUTA_tot2a',
    'AUTA_tot3a',
    'AUTA_tot4a',
    'POL_tx_objv',
    'POL_tr_capass',
    'CLI_age',
    'annee',
    'CLI_nb_contrat',
    'CLI_nb_contrat_atr',
    'CLI_nb_contrat_aup',
    'CLI_nb_contrat_bqe',
    'CLI_nb_contrat_epa',
    'CLI_nb_contrat_hab',
    'CLI_nb_contrat_mot',
    'CLI_nb_contrat_mrp',
    'CLI_nb_contrat_pf',
    'CLI_nb_contrat_pju',
    'CLI_nb_contrat_prv',
    'CLI_nb_contrat_san',
    'HAB_nb_pieces',
    'CLI_nb_enfant',
    'POL_fr',
    'AUTA_b50_n',
    'AUTA_b50_o',
    'AUTA_rm_50',
    'AUTA_rm_5060',
    'AUTA_rm_6070',
    'AUTA_rm_7090',
    'AUTA_rm_inf50',
    'AUTA_rm_sup90',
    'HAB_surf_dep',
    'HAB_inhab',
    'AUTV_nb_rc',
    'AUTV_nb_vol',
    'POL_pac',
    'POL_paj',
    'POL_pan',
    'POL_pap',
    'POL_par',
    'CLI_prod_entree',
    'HAB_qual',
    'POL_ran',
    'CLI_top_atr',
    'CLI_top_aup',
    'CLI_top_bqe',
    'CLI_top_epa',
    'CLI_top_hab',
    'CLI_top_mot',
    'CLI_top_mrp',
    'CLI_top_pf',
    'CLI_top_pju',
    'CLI_top_prv',
    'CLI_top_san',
    'HAB_top_jardin',
    'CLI_tr_age',
    'CLI_tr_ancli',
    'CLI_tr_mtvi',
    'CLI_tr_pm',
    'HAB_habit_X_qual_X_inhab',
    'POL_mtobv',
    'POL_tr_tx_objv',
    'CLI_cspf',
    'REG_zoneAxa_a',
    'REG_zoneAxa_m',
    'Random',
    'HAB_anclg',
    'CLI_sit_fam2',
    'AUTA_rm',
    'HAB_hab',
    'HAB_habit',
    'POL_delec',
    'POL_vol',
    'CLI_tr_age_X_CSPf',
    #'ddea_quant_freq_20',
    'ddea_quant_cm_10',
    #'ddem_quant_freq_20',
    #'ddem_quant_cm_20',
    'clim_quant_pp_20',
    'vol_quant_freq_20',
    'vol_quant_cm_20',
    'bdg_quant_freq_20',
    'bdg_quant_cm_20',
    'catnat_quant_pp_10',
    'fire_quant_freq_20',
    'HAB_habit_nbpieces',
    'HAB_qual_inhab',
    'POL_distrib_code',
    'Zone_inc_cm_iris',
    'Zone_inc_freq_iris',
    'zone_vol_cm_insee',
    'zone_vol_freq_insee',
]
exposure = 'anpol'
targets = ['nbsinDDE']

In [54]:
import csv
import time

import numpy as np

In [55]:
def detect_csv_separator(filename):
    """Utility function to automatically detect the separator character in a csv file."""
    with open(filename) as csvfile:
        first_line = csvfile.readline()
        return csv.Sniffer().sniff(first_line).delimiter

In [56]:
def create_data_file(out_filename, dtype, shape):
    """
    Args:
        out_filename: The name of the binary file. It must be in the same
            directory.
        dtype: The type of the numpy array.
    """
    out_file = open(out_filename, 'wb+')
    dat_file = np.memmap(out_file, dtype=dtype, shape=shape)
    return dat_file

        
def save_data_file(file, filename):
    file.flush()
    size = float(file.nbytes) / (1024 ** 2)
    print('written %s : %.3f MB' % (filename, size))

    
def create_data_file_from_list(lst, out_filename, dtype):
    """Write a list in a binary file as a numpy array.
    Args:
        lst: The list that will be written in the file.
        out_filename: The name of the binary file. It must be in the same
            directory.
        dtype: The type of the numpy array.
    """
    with open(out_filename, 'wb+') as out_file:
        dat_file = np.memmap(out_file, dtype=dtype, shape=(len(lst),))
        dat_file[:] = lst[:]
        dat_file.flush()
    size = float(dat_file.nbytes) / (1024 ** 2)
    print('written %s : %.3f MB' % (out_filename, size))

def load_data(file_path, dtype='int32', shape=None):
    return np.memmap(file_path, dtype=dtype, shape=shape)

In [57]:
from itertools import (takewhile,repeat)

def count_line(filename):
    f = open(filename, 'rb')
    bufgen = takewhile(lambda x: x, (f.raw.read(1024*1024) for _ in repeat(None)))
    return sum(buf.count(b'\n') for buf in bufgen)

In [58]:
start_time = 0

# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    global start_time
    if iteration == 0:
        start_time = time.time()
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    elapsed_time = int(time.time() - start_time)
    m = str(elapsed_time // 60).zfill(2)
    s = str(elapsed_time % 60).zfill(2)
    print('\r%s |%s| %s%% %s in %sm%ss' % (prefix, bar, percent, suffix, m, s), end = '\r')
    # Print New Line on Complete
    if iteration == total:
        print()

In [59]:
def transform(csv_filename, data_filename, features, targets, exposure):
    print('Starting data importation.')
    sep = detect_csv_separator(csv_filename)
    nb_lines = count_line(filename)
    print("Importing", '{:,}'.format(nb_lines).replace(',', ' '), "lines.")
    nb_features = len(features)
    observations = create_data_file(data_filename, np.dtype('u1'), (nb_lines, nb_features))
    exposure_data = create_data_file('./data/exposure.dat', np.dtype('float32'), (nb_lines))
    targets_data = create_data_file('./data/targets.dat', np.dtype('float32'), (nb_lines, len(targets)))
    with open(csv_filename) as csv_file:
        fields = [s.strip() for s in csv_file.readline().split(sep)]
        nb_fields = len(fields)
        features_index = [i for i in range(nb_fields) if fields[i] in features]
        if len(features_index) != nb_features:
            raise Exception("Invalid features")
        features_mapping = [{} for i in range(nb_features)]
        exposure_index = [i for i in range(nb_fields) if fields[i] == exposure]
        if len(exposure_index) != 1:
            raise Exception("Invalid Exposure field.")
        exposure_index = exposure_index[0]
        targets_index = [i for i in range(nb_fields) if fields[i] in targets]
        if len(targets_index) != len(targets):
            raise Exception("Invalid targets")
            
        for i, line in enumerate(csv_file):
            values = line.split(sep)
            if len(values) != nb_fields:
                raise Exception("Inconsistent number of fields", len(values),  "in line", i + 1, "expecting", len(fields))
            for j, index in enumerate(features_index):
                v = values[index]
                a = features_mapping[j].setdefault(v, len(features_mapping[j]))
                if a > 200:
                    raise Exception("Feature", features[j], "has too many modalities ( more than 200).")
                observations[i, j] = a
            targets_data[i, :] = [values[index] for index in targets_index]
            exposure_data[i] = values[exposure_index]
            if i % 1000 == 0 or i == nb_lines - 2:
                printProgressBar(i, nb_lines - 2, prefix = 'Progress:', suffix = 'Complete', length = 50)
    save_data_file(observations, csv_filename)
    save_data_file(targets_data, './data/exposure.dat')
    save_data_file(exposure_data, './data/targets.dat')

In [ ]:
transform(filename, './data/observations.dat', features, targets, exposure)

Starting data importation.
Importing 4 459 543 lines.


In [18]:
observations = np.memmap('./data/observations.dat', np.dtype('u1'))
exposure_data = np.memmap('./data/exposure.dat', np.dtype('float32'))
targets_data = np.memmap('./data/targets.dat', np.dtype('float32'))

In [12]:
import pandas as pd
(np.log(pd.read_csv(filename, nrows=100000)['POL_mtcapass']) / np.log(1.5)).astype('int').unique() 

array([24, 22, 18, 19, 23, 29, 17, 25, 26, 27, 21, 28, 20, 30, 16, 31, 32])

In [14]:
df = pd.read_csv(filename, nrows=10000)
for i in list(df):
    if len(df[i].unique()) < 200:
        print("'" + i + "',")

'EXT_dens_pop',
'EXT_part_respr_1p',
'EXT_part_respr_5p',
'EXT_part_secocc',
'EXT_part_surf_inf40',
'EXT_part_surf_plus100',
'EXT_part_rev_15K29k',
'EXT_part_men_lochlm',
'EXT_part_men_pror',
'EXT_part_pop_18a24a',
'EXT_part_pop_65a79a',
'EXT_part_respr_chauf_aut',
'EXT_part_respr_chauf_col',
'EXT_part_respr_chauf_ind',
'EXT_part_respr_chauf_indelec',
'EXT_part_repsr_sdb1',
'EXT_part_repsr_sdb2',
'EXT_part_lgt_co',
'EXT_part_lgt_vac',
'EXT_part_pop_15dipsup',
'EXT_part_pop_15ssdip',
'EXT_gdeville',
'AUTV_prix_sra_max',
'AUTV_prix_sra_min',
'AUTV_rm',
'AUTV_segment_max',
'AUTV_segment_min',
'CLI_banque',
'POL_cdopmrh',
'POL_fract',
'POL_cdregion',
'HAB_cdresid',
'CLI_sex',
'POL_distrib',
'AUTV_energie_es',
'AUTV_energie_go',
'AUTV_k8000_n',
'AUTV_k8000_o',
'AUTV_prix_sra_ad',
'AUTV_prix_sra_ef',
'AUTV_prix_sra_gi',
'AUTV_prix_sra_jl',
'AUTV_prix_sra_mn',
'AUTV_prix_sra_supo',
'AUTV_segment_0',
'AUTV_segment_a',
'AUTV_segment_b',
'AUTV_segment_h',
'AUTV_segment_m1',
'AUTV_segment_m2',
'A

In [32]:
features

['EXT_dens_pop',
 'EXT_part_respr_1p',
 'EXT_part_respr_5p',
 'EXT_part_secocc',
 'EXT_part_surf_inf40',
 'EXT_part_surf_plus100',
 'EXT_part_rev_15K29k',
 'EXT_part_men_lochlm',
 'EXT_part_men_pror',
 'EXT_part_pop_18a24a',
 'EXT_part_pop_65a79a',
 'EXT_part_respr_chauf_aut',
 'EXT_part_respr_chauf_col',
 'EXT_part_respr_chauf_ind',
 'EXT_part_respr_chauf_indelec',
 'EXT_part_repsr_sdb1',
 'EXT_part_repsr_sdb2',
 'EXT_part_lgt_co',
 'EXT_part_lgt_vac',
 'EXT_part_pop_15dipsup',
 'EXT_part_pop_15ssdip',
 'EXT_gdeville',
 'AUTV_prix_sra_max',
 'AUTV_prix_sra_min',
 'AUTV_rm',
 'AUTV_segment_max',
 'AUTV_segment_min',
 'CLI_banque',
 'POL_cdopmrh',
 'POL_fract',
 'POL_cdregion',
 'HAB_cdresid',
 'CLI_sex',
 'POL_distrib',
 'AUTV_energie_es',
 'AUTV_energie_go',
 'AUTV_k8000_n',
 'AUTV_k8000_o',
 'AUTV_prix_sra_ad',
 'AUTV_prix_sra_ef',
 'AUTV_prix_sra_gi',
 'AUTV_prix_sra_jl',
 'AUTV_prix_sra_mn',
 'AUTV_prix_sra_supo',
 'AUTV_segment_0',
 'AUTV_segment_a',
 'AUTV_segment_b',
 'AUTV_segm

In [51]:
len(features)

164

In [52]:
selection = [131,
146,
83,
157,
148,
138,
128,
104,
158,
32,
68,
126,
30,
151,
147,
152,
149,
122,
154,
33,]


for i in selection:
    print(features[i])

top_forfaitDDE
ddea_quant_freq_20
HAB_nb_pieces
HAB_habit_nbpieces
ddem_quant_freq_20
HAB_anclg
top_clim10M
HAB_qual
HAB_qual_inhab
CLI_sex
POL_tr_capass
REG_zoneAxa_a
POL_cdregion
vol_quant_freq_20
ddea_quant_cm_10
vol_quant_cm_20
ddem_quant_cm_20
HAB_habit_X_qual_X_inhab
bdg_quant_cm_20
POL_distrib
